### Part 1: Extract Routes and Links for Buses from 10 States.

In [ ]:
def get_routes_and_links():
    import time
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import (
        NoSuchElementException,
        StaleElementReferenceException,
        ElementClickInterceptedException
    )

    all_routes = []
    all_hrefs = []
    linklist = [
        'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile',
        'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile'
    ]

    for link in linklist:
        driver = webdriver.Chrome()
        driver.get(link)
        wait = WebDriverWait(driver, 10)

        while True:
            # Extract data from the current page
            routes = driver.find_elements(By.CSS_SELECTOR, "a[class='route']")
            route_names = [route.text for route in routes]
            hrefs = [route.get_attribute('href') for route in routes]

            all_routes.extend(route_names)
            all_hrefs.extend(hrefs)

            # Try to find the next page number and click it
            try:
                # Find the current active page number
                current_page = driver.find_element(By.CSS_SELECTOR, "div[class='DC_117_pageTabs DC_117_pageActive']")
                next_page_number = int(current_page.text) + 1

                # Find the next page button using its number
                next_page_button = driver.find_element(By.XPATH, f"//div[text()='{next_page_number}']")

                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(2)  # Wait a moment to ensure scrolling is complete

                try:
                    next_page_button.click()
                except ElementClickInterceptedException:
                    # Use JavaScript to click the button if the click is intercepted
                    driver.execute_script("arguments[0].click();", next_page_button)

                # Wait for the next page to load

            except (NoSuchElementException, StaleElementReferenceException):
                # If the next page number is not found or element reference is stale, break the loop
                break

    return {all_routes[i]: all_hrefs[i] for i in range(len(all_routes))}  # Returns route_dict, where route is the key and URL is the value

# Example usage
routes_dict = get_routes_and_links()


### Part 2: Extract Detailed Bus Information from All Routes.

In [ ]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException


def extract_bus_data(bus_element):
    """Extract data from a single bus element."""
    try:
        rating = bus_element.find_element(By.CSS_SELECTOR, "div[class='rating-sec lh-24']").text
    except NoSuchElementException:
        rating = 'New bus'

    bus_name = bus_element.find_element(By.CSS_SELECTOR, "div[class='travels lh-24 f-bold d-color']").text
    bus_type = bus_element.find_element(By.CSS_SELECTOR, "div[class='bus-type f-12 m-top-16 l-color evBus']").text
    departure_time = bus_element.find_element(By.CSS_SELECTOR, "div[class='dp-time f-19 d-color f-bold']").text
    journey_time = bus_element.find_element(By.CSS_SELECTOR, "div[class='dur l-color lh-24']").text
    arrival_time = bus_element.find_element(By.CSS_SELECTOR, "div[class='bp-time f-19 d-color disp-Inline']").text
    price = bus_element.find_element(By.CSS_SELECTOR, "div[class='seat-fare ']").text
    seats_available = bus_element.find_element(By.CSS_SELECTOR, "div[class='column-eight w-15 fl']").text

    return {
        'bus_name': bus_name,
        'bus_type': bus_type,
        'departure_time': departure_time,
        'arrival_time': arrival_time,
        'journey_time': journey_time,
        'price': price,
        'seats_available': seats_available,
        'rating': rating
    }


def scroll_amount(bus_count):
    """Determine the amount of scrolling based on the number of buses."""
    if bus_count <= 50:
        return 30
    elif bus_count <= 150:
        return 70
    elif bus_count <= 250:
        return 150
    else:
        return 230


def master_scraper(routes_dict):
    """Scrape bus data for all routes."""
    routes_zero_buses = []
    bus_details = {}

    for key in routes_dict:
        # Initialize the WebDriver
        driver = webdriver.Chrome()
        driver.get(routes_dict[key])
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "i[class='icon icon-right']"))
        )
        next_button = driver.find_element(By.CSS_SELECTOR, "i[class='icon icon-right']")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(5)

        buttons = driver.find_elements(By.CSS_SELECTOR, "div[class='button']")
        time.sleep(3)
        for button in buttons:
            driver.execute_script("arguments[0].scrollIntoView();", button)
            driver.execute_script("arguments[0].click();", button)

        # Determine the number of buses and scroll accordingly
        try:
            bus_count_element = driver.find_element(By.CSS_SELECTOR, "span[class='w-60 d-block d-found']")
            bus_count = int(re.findall(r'\b(\d+)\b', bus_count_element.text)[0])
        except NoSuchElementException:
            routes_zero_buses.append(key)
            driver.quit()
            continue

        body = driver.find_element(By.TAG_NAME, 'body')
        for _ in range(scroll_amount(bus_count)):
            body.send_keys(Keys.END)
        time.sleep(5)

        all_buses_data = driver.find_elements(By.CSS_SELECTOR, "div[class='clearfix row-one']")
        time.sleep(3)

        bus_data = [extract_bus_data(element) for element in all_buses_data]
        time.sleep(3)

        bus_details[key] = bus_data
        driver.quit()

    return bus_details, routes_zero_buses


# Example usage
bus_details, routes_zero_buses = master_scraper(routes_dict)


In [ ]:
# the cariable routes_zero_buses consists of routes that have zero buses , we are not considering them anymore 

### Part 3: Creating A Pandas DataFrame, Cleaning The Data And Then Storing It In MYSQL Database 

In [ ]:
# To extend all routes and bus data in one DataFrame
import pandas as pd
import mysql.connector

def create_and_clean_df(routes_dict, bus_details):

    df1 = pd.DataFrame()
    for route in bus_details:
        df2 = pd.DataFrame(bus_details[route])
        df2.insert(0, 'route', route)
        df2.insert(9, 'route_link', routes_dict[route])
        df1 = pd.concat([df1, df2], ignore_index=True)

    # Extracting correct data out of DataFrame
    df = df1
    df['price'] = df['price'].str.extract(r'(\d+)\s*$')
    df['price'] = df['price'].astype(int)
    df['seats_available'] = df['seats_available'].str.extract(r'^(\d+)')
    df['seats_available'] = df['seats_available'].astype(int)
    df['star_rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df['departing_time'] = pd.to_datetime(df['departure_time'], format='%H:%M').dt.time
    df['reaching_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M').dt.time
    df = df.drop(['departure_time', 'arrival_time', 'rating'], axis=1)
    df = df.fillna(0)  # New buses may have null ratings; considering 0 for such cases

    # Adding 'From' and 'To' to the DataFrame
    from_list = []
    to_list = []
    for route in df['route'].tolist():
        x, y = route.split(' to ')
        from_list.append(x)
        to_list.append(y)

    df.insert(0, 'From', from_list)
    df.insert(1, 'To', to_list)

    return df


# This function takes the DataFrame and stores it in MySQL schema
def store_in_mysql_db(df):

    df = df[['From', 'To', 'bus_name', 'bus_type', 'departing_time', 'reaching_time', 
             'journey_time', 'price', 'star_rating', 'seats_available', 'route', 'route_link']]

    # MySQL connection setup
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='12345678',
    )

    cursor = conn.cursor()
    # Create database
    cursor.execute("CREATE DATABASE IF NOT EXISTS redbusschema")
    cursor.execute("USE redbusschema")

    # Create the table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS easy_bus_data (
        `From` VARCHAR(255),
        `To` VARCHAR(255),
        bus_name VARCHAR(255),
        bus_type VARCHAR(255),
        departing_time TIME,
        reaching_time TIME,
        journey_time VARCHAR(255),
        price INT,
        star_rating FLOAT,
        seats_available INT,
        route VARCHAR(255),
        route_link VARCHAR(255)
    )
    """
    cursor.execute(create_table_query)

    # Insert DataFrame data into the table
    insert_query = """
    INSERT INTO easy_bus_data (
        `From`, `To`, bus_name, bus_type, departing_time, reaching_time, 
        journey_time, price, star_rating, seats_available, route, route_link
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Prepare the data for insertion as a list of tuples
    data_tuples = df.to_records(index=False).tolist()

    # Execute insert query for each row
    cursor.executemany(insert_query, data_tuples)

    # Commit changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()


df = create_and_clean_df(routes_dict, bus_details)
store_in_mysql_db(df)
